# Description
The code below does the following:

* Download historical position data for the INTEGRAL spacecraft
* Reads the CDF files describing WIND's trajectory
* Plots the trajectory of both spacecrafts

The trajectory data are represented by Pandas dataframes.

Parameters:

In [ ]:
from datetime import datetime, timezone, timedelta
startDate = datetime(2011, 12, 23)
endDate = datetime(2012, 1, 23, 4, 30, 0)

Configuring Plotly for offline use in notebook

In [ ]:
import plotly.offline as py
py.init_notebook_mode(connected=True)

Reading trajectory data from CSV

In [ ]:
import pandas as pd
windPosition_km = pd.read_csv('wind-pv-km-eme2000.csv', index_col=0, parse_dates=True)
integralPosition_km = pd.read_csv('integral-pv-km-eme2000.csv', index_col=0, parse_dates=True)
sunPosition_au = pd.read_csv('sun-pos-au-eme2000.csv', index_col=0, parse_dates=True)

Cropping the data

In [ ]:
windPosition_km = windPosition_km.loc[windPosition_km.index <= endDate]
windPosition_km = windPosition_km.loc[windPosition_km.index >= startDate]
integralPosition_km = integralPosition_km.loc[integralPosition_km.index <= endDate]
integralPosition_km = integralPosition_km.loc[integralPosition_km.index >= startDate]
sunPosition_au = sunPosition_au.loc[sunPosition_au.index <= endDate]
sunPosition_au = sunPosition_au.loc[sunPosition_au.index >= startDate]

# INTEGRAL
Plotting INTEGRAL's trajectory using Plotly:

In [ ]:
import plotly.graph_objs as go

trace = go.Scatter3d(
    x=integralPosition_km['x'], y=integralPosition_km['y'], z=integralPosition_km['z'],
    mode='lines',
    line=dict(
        color='#1f77b4',
        width=3
    )
)

data = [trace]


layout = go.Layout(
    scene=dict(
    aspectmode='data'
    )
)

fig = dict(data=data, layout=layout)

# py.iplot(fig)

# WIND

Plotting WIND's trajectory using Plotly

In [ ]:
import plotly.graph_objs as go

trace = go.Scatter3d(
    x=windPosition_km['x'], y=windPosition_km['y'], z=windPosition_km['z'],
    mode='lines',
    line=dict(
        #color='#1f77b4',
        width=3
    )
)

data = [trace]

layout = go.Layout(
    scene=dict(
    aspectmode='data'
    )
)

fig = dict(data=data, layout=layout)

# py.iplot(fig)

# Plotting both spacecrafts

Normalizing Sun vector and adjusting it to WIND's distance

In [ ]:
sunVector_WIND_distance = 1e6*sunPosition_au

3D plot using Plotly

In [ ]:
import plotly.graph_objs as go

earthCenter = go.Scatter3d(
    x=[0], y=[0], z=[0],
    mode='markers',
    name='Earth center'
)

windTrace = go.Scatter3d(
    x=windPosition_km['x'], y=windPosition_km['y'], z=windPosition_km['z'],
    mode='lines',
    line=dict(
        #color='#1f77b4',
        width=3
    ),
    name='WIND orbit'
)

windPosition = go.Scatter3d(
    x=[windPosition_km.iloc[-1]['x']], y=[windPosition_km.iloc[-1]['y']], z=[windPosition_km.iloc[-1]['z']],
    mode='markers',
    name='WIND current position'
)

integralTrace = go.Scatter3d(
    x=integralPosition_km['x'], y=integralPosition_km['y'], z=integralPosition_km['z'],
    mode='lines',
    line=dict(
        #color='#1f77b4',
        width=3
    ),
    name='INTEGRAL orbit'
)

integralPosition = go.Scatter3d(
    x=[integralPosition_km.iloc[-1]['x']], y=[integralPosition_km.iloc[-1]['y']], z=[integralPosition_km.iloc[-1]['z']],
    mode='markers',
    name='INTEGRAL current position'
)

sunDirection = go.Scatter3d(
    x=[0, sunVector_WIND_distance.iloc[-1]['x']], y=[0, sunVector_WIND_distance.iloc[-1]['y']], z=[0, sunVector_WIND_distance.iloc[-1]['z']],
    mode='lines',
    line=dict(
        color='#ffff00',
        width=4
    ),
    name='Sun direction'
)

data = [sunDirection, windTrace, windPosition, integralTrace, integralPosition, earthCenter]

layout = go.Layout(
    scene=dict(
    aspectmode='data'
    )
)

fig = dict(data=data, layout=layout)

py.iplot(fig)

2D plot in equator plane

In [ ]:
import plotly.graph_objs as go

earthCenter = go.Scatter(
    x=[0], y=[0],
    mode='markers',
    name='Earth center',
    marker=dict(
        size=10)
)

windTrace = go.Scatter(
    x=windPosition_km['x'], y=windPosition_km['y'],
    mode='lines',
    line=dict(
        #color='#1f77b4',
        width=3
    ),
    name='WIND orbit'
)

windPosition = go.Scatter(
    x=[windPosition_km.iloc[-1]['x']], y=[windPosition_km.iloc[-1]['y']],
    mode='markers',
    name='WIND current position'
)

integralTrace = go.Scatter(
    x=integralPosition_km['x'], y=integralPosition_km['y'],
    mode='lines',
    line=dict(
        # color='#1f77b4',
        width=3
    ),
    name='INTEGRAL orbit'
)

integralPosition = go.Scatter(
    x=[integralPosition_km.iloc[-1]['x']], y=[integralPosition_km.iloc[-1]['y']],
    mode='markers',
    name='INTEGRAL current position'
)

sunDirection = go.Scatter(
    x=[0, sunVector_WIND_distance.iloc[-1]['x']], y=[0, sunVector_WIND_distance.iloc[-1]['y']],
    mode='lines',
    line=dict(
        color='#ffff00',
        width=2
    ),
    name='Sun direction'
)

data = [sunDirection, windTrace, windPosition, integralTrace, integralPosition, earthCenter]

layout = go.Layout(
    title = f'WIND and INTEGRAL at {endDate} UTC',
    xaxis = dict(
        title='X, Earth-centered inertial, equator plane (km)'),
    yaxis = dict(
        title='Y, Earth-centered inertial, equator plane (km)'),
    scene=dict(
    aspectmode='data'
    )
)

fig = dict(data=data, layout=layout)

py.iplot(fig)

Saving plot to file

In [ ]:
import plotly.io as pio
pio.write_json(fig, '2dtraj.plotly') # As editable plotly chart
pio.write_image(fig, '2dtraj.eps') # As static EPS for LaTeX